In [1]:
import tensorflow as tf
import numpy as np
import copy

In [ ]:
class hFFLSTMRNN_tf:
    
    def __init__ (self, lenIn, lenOut, lenRec, sizeHidden, lenInRec, hiddenRec,\
                  inputs_encoded, targets, \
                  learningRate, dropout_threshold, extra_fc):
        
        # Hyper parameters
        self.lenIn          = lenIn
        self.lenOut         = lenOut
        self.lenRec         = lenRec
        self.sizeHidden     = sizeHidden
        self.lenInRec       = lenInRec
        self.hiddenRec      = hiddenRec
        self.learningRate   = learningRate
        self.dropout_threshold = dropout_threshold
        self.extra_fc         = extra_fc
        
        # input & expected output
        self.inputs_encoded = inputs_encoded;
        self.targets = targets;
        
        # parameters for inference
        self.x  = np.zeros(lenIn)  
        self.y  = np.zeros(lenOut)
        self.prob = np.zeros(lenOut)
        self.is_target = 0;
        
        self.h  = np.zeros(sizeHidden)
        self.h2 = np.zeros(int(sizeHidden/2))
        self.c  = np.zeros(hiddenRec)
        
        self.leaky_W = 0.5
        self.leaky_b = -0.1
        self.Gleaky_W = 0
        self.Gleaky_b = 0
        
        
        if(self.extra_fc == False):
            self.W  = np.zeros((lenOut,sizeHidden)) # for the last fully connected layer 
        else:
            self.W  = np.zeros((lenOut,int(sizeHidden/2)))
                           
        self.b  = np.zeros(lenOut)        
        self.GW = np.zeros_like(self.W) # Gradient, for W-update using RMSprop
        self.Gb = np.zeros_like(self.b)
        
        self.W2  = np.zeros((int(sizeHidden/2),sizeHidden)) # for the second last fully connected layer
        self.b2  = np.zeros(int(sizeHidden/2))
        self.GW2 = np.zeros_like(self.W2)
        self.Gb2 = np.zeros_like(self.b2)
        
        # for training phase 
        self.xs = np.zeros((lenRec,lenIn))       
        self.cs = np.zeros((lenRec,hiddenRec))
        self.hs = np.zeros((lenRec,sizeHidden))
        self.h2s = np.zeros((lenRec,int(sizeHidden/2)))
        self.ys = np.zeros((lenRec,lenOut))
        self.probs = np.zeros((lenRec,lenOut))
        self.is_targets = np.zeros((lenRec,1))
        
        
        # for training phase bookkeeping
        self.fg = np.zeros((lenRec,hiddenRec)) # forget gate
        self.ig = np.zeros((lenRec,hiddenRec)) # input  gate
        self.og = np.zeros((lenRec,hiddenRec)) # output gate
        self.mc = np.zeros((lenRec,hiddenRec)) # memory cell state (candidate)
        
        # LSTM class
        self.LSTM_Rec = LSTM(lenInRec+hiddenRec,hiddenRec,lenRec,learningRate)
        self.FF       = FF(lenIn-lenInRec,sizeHidden-hiddenRec,lenRec,learningRate)
                            
        # Dropout vector
        self.dvo = np.zeros((lenRec,sizeHidden));